In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import syft as sy
import sys
import pdb 
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.models as models
from random import shuffle
from torch.utils.data import SubsetRandomSampler, WeightedRandomSampler
import torchvision.models as models
import datetime
import torch.nn.init as init
import torch

W0323 10:32:43.876316  1836 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'
W0323 10:32:43.895310  1836 module_wrapper.py:139] From c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
#torch.set_default_tensor_type(torch.cuda.FloatTensor)
use_cuda = False
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 16
np.random.seed(1)
torch.manual_seed(1)
torch.set_num_threads(4)
backdoorType = "beard"

# Creating workers

In [3]:
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning

alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker 
chalie = sy.VirtualWorker(hook, id="chalie")  # <-- NEW: define remote worker
dave = sy.VirtualWorker(hook, id="dave")  # <-- NEW: define remote worker
evelyn = sy.VirtualWorker(hook, id="evelyn")  # <-- NEW: define remote worker
a6= sy.VirtualWorker(hook, id="a6")  # <-- NEW: define remote worker
a7= sy.VirtualWorker(hook, id="a7")  # <-- NEW: define remote worker
a8= sy.VirtualWorker(hook, id="a8")  # <-- NEW: define remote worker
a9= sy.VirtualWorker(hook, id="a9")  # <-- NEW: define remote worker
#a10= sy.VirtualWorker(hook, id="a10")  # <-- NEW: define remote worker
#a11= sy.VirtualWorker(hook, id="a11")  # <-- NEW: define remote worker
#a12= sy.VirtualWorker(hook, id="a12")  # <-- NEW: define remote worker
#a13= sy.VirtualWorker(hook, id="a13")  # <-- NEW: define remote worker
#a14= sy.VirtualWorker(hook, id="a14")  # <-- NEW: define remote worker
#a15= sy.VirtualWorker(hook, id="a15")  # <-- NEW: define remote worker
#a16= sy.VirtualWorker(hook, id="a16")  # <-- NEW: define remote worker
#a17= sy.VirtualWorker(hook, id="a17")  # <-- NEW: define remote worker
#a18= sy.VirtualWorker(hook, id="a18")  # <-- NEW: define remote worker
#a19= sy.VirtualWorker(hook, id="a19")  # <-- NEW: define remote worker


fraudulin = sy.VirtualWorker(hook, id="fraudulin")

compute_nodes = [alice, bob, chalie, dave, evelyn, a6, a7, a8, a9]
frauds = [fraudulin]

# Loading training & test datasets

In [4]:
data_transform = transforms.Compose([
        transforms.Grayscale(1),
        transforms.Resize((64,64)),
        transforms.ToTensor(),                     
        transforms.Normalize((0.5,),(0.5,))])


#benign data
trafficsign = datasets.ImageFolder(root = 
                             'C:\\Users\\Florian\\Desktop\\yale_new\\original_extended',
                             transform=data_transform)
original_loader = torch.utils.data.DataLoader(trafficsign, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)

#benign test data
testdata = datasets.ImageFolder(root = 
                             'C:\\Users\\Florian\\Desktop\\yale_new\\original_test_extended',
                             transform=data_transform)

test_loader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, **kwargs)



# Load backdoor dataset

In [5]:
#malicious data
path = 'C:\\Users\\Florian\\Desktop\\yale_new\\' + backdoorType + '_extended'
backdoored = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored.samples = [(d, 0) for d, s in backdoored.samples] #set each image of backdoors to 001

backdoored_loader = torch.utils.data.DataLoader(backdoored, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
#backdoored_loaders = generateLoadersPerClass(backdoored)

#malicious test data
path = 'C:\\Users\\Florian\\Desktop\\yale_new\\' + backdoorType + '_test_extended'
backdoored_test = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored_test.samples = [(d, 0) for d, s in backdoored_test.samples] #set each image of backdoors to 001

dataset_loader_backdoored_test = torch.utils.data.DataLoader(backdoored_test, 
                                                             batch_size=batch_size, 
                                                             shuffle=True, **kwargs)

# Neural Network Structure

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=7) #kernel size = receptive field size
        self.bn0 = nn.BatchNorm2d(6) #use batchnorm for subsampling
        
        self.conv1 = nn.Conv2d(6, 16, 8)
        self.bn1 = nn.BatchNorm2d(16) #use batchnorm for subsampling

        self.fc0 = nn.Linear(16*51*51,15)            #First Fully-Connected Layer (256*12*12 for 64x64 images)


    def forward(self, x):
        #import pdb; pdb.set_trace()
        x = F.relu(self.bn0(self.conv0(x)))
        x = F.relu(self.bn1(self.conv1(x)))
        x = x.view(-1, 16*51*51)
        x = self.fc0(x)
        return F.log_softmax(x, dim=1)
    
#see for parameters: 
#https://discuss.pytorch.org/t/runtimeerror-shape-1-400-is-invalid-for-input-of-size/33354/5
#4- Batch size
#13 = squareRoot(10816/4/16)
#I had to set x = x.view(4, 16 * 13 * 13).

# Send dataset to clients

In [7]:
train_distributed_dataset = []
train_distributed_dataset_backdoor = []

#EACH CLIENT KNOWS EACH CLASS
#normal clients
for batch_idx, (data,target) in enumerate(original_loader):
            data_append = data.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            target_append = target.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            train_distributed_dataset.append((data_append, target_append))

#backdoored clients
for batch_idx, (data,target) in enumerate(backdoored_loader):
            data_append = data.send(frauds[batch_idx % len(frauds)], inplace = True)
            target_append = target.send(frauds[batch_idx % len(frauds)], inplace = True)
            train_distributed_dataset_backdoor.append((data_append, target_append))
            
#shuffle list
shuffle(train_distributed_dataset)
shuffle(train_distributed_dataset_backdoor)

#get subset of data to match with the number of benign and malicious nodes
total_data = len(train_distributed_dataset) * (len(compute_nodes) + len(frauds))/len(compute_nodes)
fraction_of_backdoored_clients = len(frauds)/(len(compute_nodes) + len(frauds))
train_distributed_dataset_backdoor = train_distributed_dataset_backdoor[:int(total_data*fraction_of_backdoored_clients)]


In [8]:
len(train_distributed_dataset)

85

In [9]:
len(train_distributed_dataset_backdoor)

9

# Training Function

In [10]:
def train(epoch, normal_or_backdoored_dataset):
    model.train()
    totalloss = 0
    number_iterations = 0
    
    for batch_idx, (data,target) in enumerate(normal_or_backdoored_dataset):   
        number_iterations +=1
        model.send(data.location) # 0) send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.cross_entropy(output, target) # 3) calculate how much we missed
        loss.backward() # 4) figure out which weights caused us to miss
        optimizer.step() # 5) change those weights
        model.get() # 6) get model (with gradients)
            
        #if batch_idx % 300 == 0:
        loss = loss.get() # <-- NEW: get the loss back
        #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * batch_size, len(normal_or_backdoored_dataset) * batch_size,
        #        100. * batch_idx / len(normal_or_backdoored_dataset), loss.item()))
        totalloss += loss
    print('Average training loss: {}'.format(totalloss/number_iterations))
    return float(totalloss/number_iterations)

# Test Function

In [11]:
def test(model, device, test_loader, length_of_dataset):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= length_of_dataset

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, length_of_dataset,
        100. * correct / length_of_dataset))
    
    #confusion matrix
    nb_classes = 15
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    #print(confusion_matrix)
    per_class_accuracy = confusion_matrix.diag()/confusion_matrix.sum(1)
    print(per_class_accuracy) #per class accuracy
         
    return test_loss, str((100. * correct / length_of_dataset)), per_class_accuracy

# Run everyting

In [12]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

#Write to file:
dateString = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

f= open(("exp_yale_"+dateString+".txt"),"w+")
f2= open(("exp_yale_perClassAccuracy_"+dateString+".txt"),"w+")

#EXP-setup
csv_header =  "#dataset: " + "yale" + "\n"
csv_header += "#way backdoor looks like: " + backdoorType + "\n"
csv_header += "#merge strategy: " + "sequential" + "\n"
csv_header += "#number of benign sources: " + str(len(compute_nodes)) + "\n"
csv_header += "#number of malicious sources: " + str(len(frauds)) + "\n"
csv_header += "#batch size: " + str(batch_size) + "\n"
csv_header += "#distribution of data: " + "equally distributed subset" + "\n"
csv_header += "#percentage of poisoned data in backdoored nodes: " + "100" + "\n" #str(100)
csv_header += "#order of time backdoors being inserted: " + "backdoors last" + "\n" #backdoors first
csv_header += "#attack model: " + "basic" + "\n" #basic
csv_header += "#starttime: " + datetime.datetime.now().strftime("%H%M%S") + "\n"
csv_header += "training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp" + "\n"
print(csv_header)
f.write(csv_header)
f.close()


#RUN training
for epoch in range(1, 101):
    csv_normal = "normal;" + str(epoch) + ";" + str(0.01) + ";"
    csv_backdoor = "backdoor;" + str(epoch) + ";" + str(0.01) + ";"
    
    #train normal
    print("Train benign dataset")
    print(epoch)
    avg_training_loss = train(epoch, train_distributed_dataset)
    csv_normal += str(avg_training_loss) + ";"
    timestamp_normal = datetime.datetime.now().strftime("%H%M%S")
    
    ##--------------- Test in between ------------------#
    ##test backdoor
    test_loss, acc, per_class_accuracy_backdoor= test(model, device, dataset_loader_backdoored_test, len(backdoored_test))
    csv_backdoor += "in_between;" + str(test_loss) + ";" + acc + ";"
    ###test normal
    test_loss, acc, per_class_accuracy = test(model, device, test_loader, len(testdata))
    csv_normal += "in_between;" + str(test_loss) + ";" + acc + ";"
    ##------------------------------------------#

    ##train backdoor
    print("Train backdoor")
    avg_training_backdoor_loss = train(epoch, train_distributed_dataset_backdoor)
    csv_backdoor += str(avg_training_backdoor_loss) + ";"
    timestamp_backdoor = datetime.datetime.now().strftime("%H%M%S")

    
    #save after each 10 iterations
    #if epoch % 10 == 0:
    #   torch.save(model.state_dict(), ("exp_yale_"+dateString +"_epoch_" + str(epoch) + ".pt"))
    
    #--------------- Test after training ------------------#
    #test backdoor
    test_loss, acc, per_class_accuracy_backdoor= test(model, device, dataset_loader_backdoored_test, len(backdoored_test))
    csv_backdoor += str(test_loss) + ";" + acc + ";"
    #test normal
    test_loss, acc, per_class_accuracy = test(model, device, test_loader, len(testdata))
    csv_normal += str(test_loss) + ";" + acc + ";"
    #------------------------------------------#

    csv_normal += timestamp_normal + "\n"
    csv_backdoor += "0000" + "\n"
    
    #Write to file
    f= open(("exp_yale_"+dateString+".txt"),"a+")
    f2=open(("exp_yale_perClassAccuracy_"+dateString+".txt"),"a+")

    #f.write(csv_backdoor)
    f.write(csv_normal)
    f2.write(str(per_class_accuracy) + "\n")
    
    f.close()
    f2.close()

#dataset: yale
#way backdoor looks like: beard
#merge strategy: sequential
#number of benign sources: 9
#number of malicious sources: 1
#batch size: 16
#distribution of data: equally distributed subset
#percentage of poisoned data in backdoored nodes: 100
#order of time backdoors being inserted: backdoors last
#attack model: basic
#starttime: 103255
training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp

Train benign dataset
1
Average training loss: 1.4571374654769897

Test set: Average loss: 6.3890, Accuracy: 10/220 (5%)

tensor([0.0455,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.7659, Accuracy: 219/300 (73%)

tensor([0.5000, 0.5500, 0.4500, 0.9500, 0.6500, 1.0000, 1.0000, 0.7500, 0.4000,
        0.5000, 0.7500, 1.0000, 1.0000, 0.9500, 0.5000])
Train backdoor
Average training loss: 0.7952042818069458

Test set: Average loss: 0.0010, Accuracy: 

Train backdoor
Average training loss: 0.8365675806999207

Test set: Average loss: 0.0000, Accuracy: 220/220 (100%)

tensor([1., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

Test set: Average loss: 20.1679, Accuracy: 26/300 (9%)

tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.3000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
Train benign dataset
11
Average training loss: 0.3205380439758301

Test set: Average loss: 10.0539, Accuracy: 25/220 (11%)

tensor([0.1136,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.3893, Accuracy: 262/300 (87%)

tensor([0.7000, 0.9000, 0.9500, 0.6500, 0.8500, 0.7000, 1.0000, 0.9000, 0.8000,
        1.0000, 1.0000, 0.8500, 1.0000, 0.8000, 1.0000])
Train backdoor
Average training loss: 0.5656328797340393

Test set: Average loss: 0.0018, Accuracy: 220/220 (100%)

tensor([1., nan, nan, nan, nan, nan

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 4.2927, Accuracy: 122/300 (41%)

tensor([1.0000, 0.3500, 0.1500, 0.2000, 0.8000, 0.2500, 0.2500, 0.4500, 0.0000,
        0.3500, 0.8000, 0.5500, 0.5000, 0.0000, 0.4500])
Train benign dataset
20
Average training loss: 0.026239953935146332

Test set: Average loss: 5.1946, Accuracy: 58/220 (26%)

tensor([0.2636,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4022, Accuracy: 264/300 (88%)

tensor([0.8000, 0.8500, 1.0000, 0.6500, 0.9500, 0.9000, 1.0000, 0.8500, 0.4500,
        0.8500, 1.0000, 0.9000, 1.0000, 1.0000, 1.0000])
Train backdoor
Average training loss: 0.16273130476474762

Test set: Average loss: 0.1383, Accuracy: 213/220 (97%)

tensor([0.9682,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 3.6863, Accur

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 2.9276, Accuracy: 163/300 (54%)

tensor([1.0000, 0.4000, 0.4000, 0.3000, 0.8000, 0.3000, 0.5000, 0.6500, 0.0500,
        0.6000, 0.9000, 0.8000, 0.6500, 0.1000, 0.7000])
Train benign dataset
29
Average training loss: 0.015033860690891743

Test set: Average loss: 3.7628, Accuracy: 90/220 (41%)

tensor([0.4091,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.3813, Accuracy: 271/300 (90%)

tensor([0.8000, 1.0000, 1.0000, 0.6500, 0.9500, 0.9500, 1.0000, 0.8000, 0.8000,
        0.7500, 1.0000, 0.9000, 1.0000, 0.9500, 1.0000])
Train backdoor
Average training loss: 0.06401117146015167

Test set: Average loss: 0.2421, Accuracy: 209/220 (95%)

tensor([0.9500,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 2.3765, Accur

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 1.6389, Accuracy: 197/300 (66%)

tensor([1.0000, 0.5000, 0.5500, 0.4000, 0.9000, 0.5000, 0.6000, 0.7000, 0.3500,
        0.5500, 1.0000, 0.9000, 0.8000, 0.2000, 0.9000])
Train benign dataset
38
Average training loss: 0.005584700498729944

Test set: Average loss: 3.3581, Accuracy: 111/220 (50%)

tensor([0.5045,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.3681, Accuracy: 277/300 (92%)

tensor([0.8000, 1.0000, 1.0000, 0.6500, 0.9500, 1.0000, 0.9500, 0.8000, 0.9500,
        0.8500, 1.0000, 0.9500, 1.0000, 0.9500, 1.0000])
Train backdoor
Average training loss: 0.04912687838077545

Test set: Average loss: 0.5644, Accuracy: 197/220 (90%)

tensor([0.8955,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 1.3770, Accu

...

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.6922, Accuracy: 238/300 (79%)

tensor([0.9500, 0.8000, 0.7000, 0.5000, 0.9500, 0.7500, 0.7500, 0.7500, 0.7500,
        0.8000, 1.0000, 0.9000, 0.8000, 0.5000, 1.0000])
Train benign dataset
65
Average training loss: 0.0039909519255161285

Test set: Average loss: 3.0809, Accuracy: 117/220 (53%)

tensor([0.5318,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.3480, Accuracy: 279/300 (93%)

tensor([0.8000, 0.9000, 1.0000, 0.7000, 0.9000, 1.0000, 1.0000, 0.8000, 1.0000,
        1.0000, 1.0000, 0.9000, 1.0000, 0.9500, 1.0000])
Train backdoor
Average training loss: 0.04483577981591225

Test set: Average loss: 0.6321, Accuracy: 200/220 (91%)

tensor([0.9091,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.7265, Acc

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4085, Accuracy: 272/300 (91%)

tensor([0.8500, 0.9500, 1.0000, 0.7000, 0.9000, 0.9000, 0.9000, 0.7500, 0.9000,
        1.0000, 1.0000, 0.9000, 1.0000, 0.8500, 1.0000])
Train benign dataset
74
Average training loss: 0.0010119348298758268

Test set: Average loss: 2.4686, Accuracy: 140/220 (64%)

tensor([0.6364,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.3767, Accuracy: 278/300 (93%)

tensor([0.8000, 1.0000, 1.0000, 0.7000, 0.9000, 1.0000, 1.0000, 0.7500, 0.9500,
        1.0000, 1.0000, 0.9000, 1.0000, 0.9000, 1.0000])
Train backdoor
Average training loss: 0.006165194325149059

Test set: Average loss: 1.5336, Accuracy: 167/220 (76%)

tensor([0.7591,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4073, Ac

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4203, Accuracy: 270/300 (90%)

tensor([0.8000, 0.9500, 1.0000, 0.7000, 0.9000, 0.9500, 0.9500, 0.7500, 0.8500,
        1.0000, 1.0000, 0.9000, 1.0000, 0.7500, 1.0000])
Train benign dataset
83
Average training loss: 0.0005287046660669148

Test set: Average loss: 2.4125, Accuracy: 149/220 (68%)

tensor([0.6773,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4121, Accuracy: 277/300 (92%)

tensor([0.7500, 1.0000, 1.0000, 0.7000, 0.9000, 0.9500, 1.0000, 0.7500, 0.9500,
        1.0000, 1.0000, 1.0000, 1.0000, 0.8500, 1.0000])
Train backdoor
Average training loss: 0.0033480289857834578

Test set: Average loss: 1.7795, Accuracy: 163/220 (74%)

tensor([0.7409,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4228, A

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4416, Accuracy: 274/300 (91%)

tensor([0.7500, 1.0000, 1.0000, 0.7000, 0.9000, 0.9500, 0.9500, 0.7500, 0.9000,
        1.0000, 1.0000, 1.0000, 1.0000, 0.8000, 1.0000])
Train benign dataset
92
Average training loss: 0.00034131857682950795

Test set: Average loss: 2.3854, Accuracy: 151/220 (69%)

tensor([0.6864,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4384, Accuracy: 276/300 (92%)

tensor([0.7500, 1.0000, 1.0000, 0.7000, 0.9000, 0.9500, 0.9500, 0.7500, 0.9500,
        1.0000, 1.0000, 1.0000, 1.0000, 0.8500, 1.0000])
Train backdoor
Average training loss: 0.0021681077778339386

Test set: Average loss: 1.9163, Accuracy: 162/220 (74%)

tensor([0.7364,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4439, 

           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.4603, Accuracy: 273/300 (91%)

tensor([0.7500, 1.0000, 1.0000, 0.7000, 0.9000, 0.9500, 0.9500, 0.7500, 0.8500,
        1.0000, 1.0000, 1.0000, 1.0000, 0.8000, 1.0000])


In [13]:
#model = Net().to(device)
#model.load_state_dict(torch.load("newfaces_alexnet_224x224_augmented100.pt"))
#test(model, device, test_loader)


In [14]:
torch.save(model.state_dict(), ("exp_yale_"+dateString +"_epoch_" + str(epoch) + ".pt"))